Spark x Twitter API v2 Stream (Essential access)

References:
Udemy - Spark and Python for Big Data with PySpark

In [ ]:
import tweepy
from tweepy import Stream

In [ ]:
import socket
import json

In [ ]:
consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_secret = 'xxx'

In [ ]:
class TwitterStreamer(Stream):
    def __init__(self, *args, c_socket):
        super().__init__(*args)
        self.client_socket = c_socket
        
    def on_data(self, data):
        try:
            msg = json.load(data)
            print(msg['text'].encode('utf-8'))
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True
        
        except BaseExpetion as e:
            print("ERROR ", e)
        return True
    
    def on_error(self, status):
        print(status)
        return True

In [ ]:
def sendData(c_socket):    
    #Parameters for constructing a Stream instance are changed from when the Udemy's video was recorded (The version I am using here is Tweepy 4.5), and below don't work anymore
    #twitter_stream = TwitterStreamer(consumer_key,
    #                               consumer_secret,
    #                               access_token,
    #                               access_secret,
    #                               c_socket=c_socket)
    
    #myStreamListener = TwitterStreamer(c_socket)
    #twitter_stream = Stream(auth, myStreamListener)
    
    twitter_stream = Stream(consumer_key,consumer_secret, access_token, access_secret)
    
    twitter_stream.filter(track=['guitar'])

In [ ]:
if __name__ == "__main__":
    s = socket.socket()
    host = '127.0.0.1'
    port = 7784
    s.bind((host, port))
    
    s.listen(5)
    print("listening on port {}".format(port))
    
    c, addr = s.accept()
    print( "Received request from: " + str( addr ) )
    
    sendData(c)

Above code gave me a 403 error at "twitter_stream.filter()", and I wasn't able to stream Tweets. According to the document below, it seems it was due to me having only the Essential access.

References:
https://docs.tweepy.org/en/latest/faq.html#why-am-i-encountering-a-403-forbidden-error
```
If you have Essential access to the Twitter API, you won’t be able to access Twitter API v1.1. This includes all API methods and Stream.
You can use Twitter API v2 with Client or apply for Elevated access.
```

So instead, I will try using Twitter API v2 with Client.

Try something similar to above with Twitter API v2 Client's search_recent_tweets function.
Since this isn't a stream and gets tweets from the past seven days based on the specified query, I will get some of the same tweets. (But for the purpose of this study, it is fine.)

In [1]:
from tweepy import Client
import socket
import time

In [2]:
bearer_token = 'xxx'
consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_secret = 'xxx'

In [3]:
def get_Tweets(t_c, query):
    return t_c.search_recent_tweets(query=query)

In [4]:
def send_Tweets(c_socket, tweets):
    try:
        for t in tweets:
            msg = t.text + "\n"
            print(msg)
            c_socket.send(msg.encode('utf-8'))
        
        return True

    except:
        print("ERROR !")
    return True

In [5]:
if __name__ == "__main__":
    s = socket.socket()
    host = '127.0.0.1'
    port = 7755
    s.bind((host, port))
    
    s.listen(5)
    print("listening on port {}".format(port))
    
    c, addr = s.accept()
    print("Received request from: " + str( addr ))
    
    twitter_client = Client(bearer_token, consumer_key, consumer_secret, access_token, access_secret)
    
    query = '(#20thAnniversary OR #HarryPotter) lang:en'
    
    while True:
        tweets = get_Tweets(twitter_client, query)
        send_Tweets(c, tweets.data)
        
        time.sleep(10)

listening on port 7755
Received request from: ('127.0.0.1', 33288)
RT @TheWoodShop5: This awesome ravenclaw flask is is just $15! Can also make them with different designs! This makes a great gift for Harry…

RT @selfcovehp: I don’t ship BlaisexRon but this artwork by alek.dar on insta is well good 🤪
#art #harrypotter #fanart https://t.co/AoPjUoF…

RT @MagicalShipPod: ✨Was there a specific fic or piece of fanart that made you go: "Omg this ship is EVERYTHING." 

#HarryPotter #FanFictio…

RT @LadyKalsifer: Drawing ships ❤
#otp #digitalart #fanart #souleater #goodomens #misskoboyashisdragonmaid #harrypotter #drarry #ineffableh…

Great Tour: Warner Bros. Studio Tour London - The Making of Harry Potter https://t.co/IY0iUMkOdD #Travel #TravelVideo #Vacation #VacationVideo #TravelPics #Photography #London #harrypotter #WarnerBros #StudioTour #hogwarts #potterhead #wizardingworld #harrypotterworld #hp #vlog

RT @TheWoodShop5: This awesome ravenclaw flask is is just $15! Can also make them wit

KeyboardInterrupt: 

Although above isn't "streaming", I can still continuously get tweets by putting them in while loop.

Now, assuming the above is streaming, I will use PySpark to get this "streaming" of data and visualize it in the "SparkStreaming_stream.ipynb".